## 1.4. DQN
接下来我们来研究Q-Learning。我们首先来介绍两种值函数：状态值函数$v_{\pi}(s)$和状态行动值函数$q_{\pi}(s,a)$。在业界也将这两个函数称为Critic，即评判者，可以用于评价策略$\pi$的好坏，并据此对策略进行改进，也就是著名的Q-Learning，以及其对应的深度学习版本DQN。
### 1.4.1. 状态值函数
状态值函数$v_{\pi}(s)$表求在状态$s$可以获得的累积奖励的期望值：
$$
v_{\pi}=E(G_{t}|S_{t}=s)
$$
#### 1.4.1.1. Monte Carlo法
我们首先让Agent与环境互动$N$次，如玩$N$局游戏，对于每一局游戏：
$$
r_{1}^{1}, s_{1}^{1}, a_{1}^{1}, r_{2}^{1}, s_{2}^{1}, a_{2}^{1}, ...,r_{T_{1}}^{1},s_{T_{1}}^{1},a_{T_{1}}^{1}
$$
根据累称奖励定义：
$$
G_{t}=\sum_{k=0}^{\infty}\gamma ^{k} R_{t+k+1}
$$
我们可以求出每个时刻累积奖励：
$$
G_{1}^{1},G_{2}^{1},...,G_{T_{1}}^{1}
$$
与每一时刻状态$s_{t}$进行组合，可以形成如下数据集：
$$
(s_{1}^{1},G_{1}^{1}),(s_{2}^{1},G_{2}^{1}),...,(s_{T_{1}}^{1},G_{T_{1}}^{1})
$$
因此所有$N$个Trajectory形成一个数据集：
$$
(s_{1}^{1},G_{1}^{1}),(s_{2}^{1},G_{2}^{1}),...,(s_{T_{1}}^{1},G_{T_{1}}^{1}) \\
(s_{1}^{2},G_{1}^{2}),(s_{2}^{2},G_{2}^{2}),...,(s_{T_{1}}^{2},G_{T_{1}}^{2}) \\
...... \\
(s_{1}^{N},G_{1}^{N}),(s_{2}^{N},G_{2}^{N}),...,(s_{T_{1}}^{N},G_{T_{1}}^{N})
$$
对于每个样本，$s_{i}^{j}$代表在第$j$局中，第$i$个时间点的状态，为一个向量，如图像画面，$G_{i}^{j}$为其所对应的累积奖励，为一个标量数值。我们可以将其视为一个回归（Regression）任务，输入为向量$s_{i}^{j}$，输出值为$G_{i}^{j}$。

#### 1.4.1.2. Temporal Difference法
我们让Agent与环境互动，可以得到如下的Trajectory片断：
$$
......, s_{t}, a_{t}, r_{t+1}, s_{t+1}, ......
$$
根据状态值函数的定义可得：
$$
v_{\pi}(s_{t})=r_{t+1} + v_{\pi}(s_{t+1})
$$
将上式进行整理可得：
$$
v_{\pi}(s_{t}) - v_{\pi}(s_{t+1}) = r_{t+1}
$$
采用如下网络结构：
![TD法](./images/chp_01_04_01.png)

#### 1.4.1.3. MV vs TD
通过Monte Carlo和Temporal Difference方法，都可以通过回归算法（Regression）求出状态值函数。对于MC方法，如果数据量足够大，可以得到均值，但是会有比较大的方差。而TD方法则正好相反，其不具有逼近均值的能力，但是估计结果的方差较小。在实际使用中，TD方法会更常用一些。

### 1.4.2. 状态行动值函数
对于$q_{\pi}(s, a)$，我们在状态$s$时采用行动$a$，然后在其后的过程中，仍然使用策略$\pi$来选择行动，对于行动为离散型时：
![q](./images/chp_01_04_02.png)
如果行动是连续型的情况，则可以将状态$s$与所采取的行动一起，输入到网络，如下所示：
![q continue](./images/chp_01_04_03.png)

### 1.4.3. Q-Learning


下面我们来看Q-Learning的推导过程。我们假设有策略$\pi ^{*}$，满足：
$$
\pi ^{*}(s) = \arg \max_{a \in \mathcal{A}(s)} q_{\pi}(s, a) \\
\forall s \in \mathcal{S}: v_{\pi ^{*}}(s) \geqslant v_{\pi}(s)
$$
上式表明，在策略$\pi ^{*}$下，在状态$s$时，所采取的行动是使$q_{\pi}(s, a)$的值达到最大时的行动$a$。在策略$\pi ^{*}$下，对于所有的状态$s$，所取得的状态值函数的值均大于或等于在其他策略下状态值函数的值。
状态值函数可以视为是在状态$s$下根据策略$\pi$采取行动所获得的累积奖励，其一定小于或等于在状态$s$，采用最佳策略$\pi {*}$时采取的行动所获得的累积奖励，如下所示：
$$
v_{\pi}(s)=q_{\pi}(s, \pi(s)) \leqslant \max_{a \in \mathcal{A}(s)} q_{\pi}(s, a)=q_{*}(s, \pi ^{*}(s))
$$
将上式写为$q_{\pi}(s,a)$的定义式，如下所示：
$$
v_{\pi}(s) \leqslant q_{*}\Big( s, \pi ^{*}(s) \Big) = E \Big( r_{t+1} + v_{\pi}(s_{t+1})|S_{t}=s, A_{t}=\pi ^{*}(s) \Big)
$$
该值肯定小于在$t+1$时刻也采用最佳策略$\pi ^{*}$时采取的行动，如下所示：
$$
v_{\pi}(s_{t+1}) \leqslant  E \Big( r_{t+1} + v_{\pi}(s_{t+1})|S_{t}=s_{t}, A_{t}=\pi ^{*}(s_{t+1}) \Big) \\
\leqslant E \bigg( r_{t+1} + q_{\pi}(s_{t+1}, \pi ^{*}(s_{t+1}) | S_{t+1}=s_{t+1}, A_{t+1}=\pi ^{*}(s_{t+1}) \bigg) \\
= E \bigg( r_{t+1} + r_{t+2} + v_{\pi}(s_{t+2}) | S_{t+1}=s_{t+1}, A_{t+1}=\pi ^{*}(s_{t+1}) \bigg)  \\
...... \\
= E(r_{t+1} + r_{t+2} + ... + r_{T}) \\
= v_{\pi ^{*}}(s_{t}) = v_{*}(s_{t})
$$

#### 1.4.3.1. TD法
在实际的Q-Learning中，我们采用类似TD的方法。
我们首先让Agent与环境进行互动，得到如下Trajectory片断：
$$
..., s_{t}, a_{t}, r_{t+1}, s_{t+1}, ...
$$
根据定义有如下公式：
$$
q_{\pi}(s_{t}, a_{t})= r_{t+1} + q_{\pi}(s_{t+1}, \pi (s_{t+1}))
$$
式中$\pi (s_{t+1})$表示在状态$s_{t+1}$下由策略$\pi$来选择合适的行动。
我们采用如下的网络结构：
![target network](./images/chp_01_04_04.png)
在初始时，图中的Target NN与NN具有相同的参数$\boldsymbol{\theta}_{\pi}^{0}$，然后按照回归算法（Regression）来更新图中上面NN的参数，但是保持下面Target NN的参数不变，当运行$N$（超参数）次之后，用上面网络的参数$\boldsymbol{\theta}_{\pi}^{N}$去更新Target NN的参数。

#### 1.4.3.2. 探索
我们注意到，在Q-Learning中，策略$\pi$为：$a=\arg \max_{a \in \mathcal{A}(s)} q_{\pi}(s,a)$，其实际上没有探索的过程。在实际的算法中，我们通常会引入探索的成份。
* epsilon Grady算法
$$
a = \begin{cases} \arg \max_{a \in \mathcal{A}(s)} q_{\pi}(s, a), \quad p=1-\epsilon \\
random, \quad p=\epsilon
\end{cases}
$$
即其以$1-\epsilon$的概率取策略给定的行动，以$\epsilon$的概率随机取值。$\epsilon$的值随着训练的进行逐渐减小。
* Boltzman Exploration
我们定义取策略指定行动的概率为：
$$
p(a|s) = \frac{ \exp \big( q_{\pi}(s,a) \big) }{\sum_{a' \in \mathcal{A}(s)} \exp \big( q_{\pi}(s,a') \big) }
$$
在其他情况下随机取值。

#### 1.4.3.3. Experience Buffer
初始时让Agent与环境进行$N$次互动，得到如下Trajectory：
$$
\tau ^{1} = \{ s_{1}^{1},a_{1}^{1},r_{2}^{1},s_{2}^{1},a_{2}^{1},...,r_{T}^{1},s_{T}^{1},a_{T}^{1} \} \\
\tau ^{2} = \{ s_{1}^{2},a_{1}^{2},r_{2}^{2},s_{2}^{2},a_{2}^{2},...,r_{T}^{2},s_{T}^{2},a_{T}^{2} \} \\
...... \\
\tau ^{N} = \{ s_{1}^{N},a_{1}^{N},r_{2}^{N},s_{2}^{N},a_{2}^{N},...,r_{T}^{N},s_{T}^{N},a_{T}^{N} \}
$$
我们把上述Trajectory的数据，整理成如下所示的元组形式：
$$
s_{t}^{n},a_{t}^{n}, r_{t+1}^{n}, s_{t+1}^{n}
$$
其代表在第$n$个Trajectory的第$t$时刻，其状态为$s_{t}^{n}$，采取行动$a_{t}^{n}$，其收到环境给的奖励信号$r_{t+1}^{n}$，同时环境转移到状态$s_{t+1}^{n}$。
我们得到如下元组的集合称之为Replay Buffer：
$$
s_{1}^{1}, a_{1}^{1}, r_{2}^{1}, s_{2}^{1} \\
s_{2}^{1}, a_{2}^{1}, r_{3}^{1}, s_{3}^{1} \\
s_{3}^{1}, a_{3}^{1}, r_{4}^{1}, s_{4}^{1} \\
...... \\
s_{T_{1}-1}^{1}, a_{T_{1}-1}^{1}, r_{T_{1}}^{1}, s_{T_{1}}^{1} \\
s_{1}^{2}, a_{1}^{2}, r_{2}^{2}, s_{2}^{2} \\
s_{2}^{2}, a_{2}^{2}, r_{3}^{2}, s_{3}^{2} \\
s_{3}^{2}, a_{3}^{2}, r_{4}^{2}, s_{4}^{2} \\
...... \\
s_{T_{2}-1}^{2}, a_{T_{2}-1}^{2}, r_{T_{2}}^{2}, s_{T_{2}}^{2} \\
...... \\
s_{1}^{N}, a_{1}^{N}, r_{2}^{N}, s_{2}^{N} \\
s_{2}^{N}, a_{2}^{N}, r_{3}^{N}, s_{3}^{N} \\
s_{3}^{N}, a_{3}^{N}, r_{4}^{N}, s_{4}^{N} \\
...... \\
s_{T_{N}-1}^{N}, a_{T_{N}-1}^{N}, r_{T_{N}}^{N}, s_{T_{N}}^{N}
$$
接下来我们就可以从Experience Buffer中Sample出batch_size个样本，运行Target Network章节的算法，得到新的策略$\pi '$。
再拿新策略$\pi '$与环境互动$m$次，同样生成上面所述的元组，并添加到Experience Buffer中。如果Experience Buffer已满，则将最老的元组挤出Experience Buffer。
采用Experience Buffer主要有以下两点好处：
* 节省与环境互动的时间：通常Agent与环境互动是非常耗时的操作，采用事先录制好的Experience Buffer可以大量节约时间；
* 训练样本具有多样性：由于采用不同策略来进行训练，网络性能会更好；

### 1.4.4. 传统DQN算法
算法原理如下所示：
* 初始化NN和Target NN：初先初始化NN网络$Q_{\pi}$和Target NN网络$\hat{Q}_{\pi}$，并且令二者的参数相等，即$Q_{\pi}=\hat{Q}_{\pi}$；
* 让Agent与环境进行$K$个Episode：
  * 在第$k$个Episode的第$t$个时间点：
    * 其在$1-\epsilon$的概率下使用下式选出的行动，在$\epsilon$概率下随机选择：
    $$
    a_{t}^{k} = \arg \max_{ a_{t}^{k} \in \mathcal{A}(s_{t}^{k}) } q_{\pi}(s_{t}^{k}, a_{t}^{k})
    $$
    实际的求解方法为将$s_{t}^{k}$与所有的$a_{t}^{'k} \in \mathcal{A}(s_{t}^{k})$的行动对作为输入，输入到$Q_{\pi}$网络中，计算出相应的$q_{\pi}(s_{t}^{k}, a_{t}^{'k})$的值，最终的$a_{t}^{k}$为其中可以取得$q_{\pi}(s_{t}^{k}, a_{t}^{'k})$最大值的那个$a_{t}^{'k}$；
    * Agent会得到奖励$r_{t+1}^{k}$，并且环境会转移到新状态$s_{t+1}^{k}$；
    * 将元组$s_{t}^{k}, a_{t}^{k}, r_{t+1}^{k}, s_{t+1}^{k}$保存到Experience Buffer中；
    * 如果Experience Buffer小于batch_size大小，则直接返加循环顶部，进行第$k$个episode的第$t+1$时刻。如果Experience Buffer已经达到最大容量，则将最老的元组挤出，将当前元组加入；
    * 从Experience Buffer中Sample出batch_size个元组；
    * 将$s_{t}^{k}$和$a_{t}^{k}$输入到网络$Q_{\pi}$中，得到$q_{\pi}(s_{t}^{k}, a_{t}^{k}$；
    * 通$s_{t+1}^{k}$与所有的$a_{t+1}^{'k} \in \mathcal{A}(s_{t+1}^{k})$的行动对作为输入，输入到$Q_{\pi}$网络中，计算出相应的$q_{\pi}(s_{t+1}^{k}, a_{t+1}^{'k})$的值，最终的$a_{t+1}^{k}$为其中可以取得$q_{\pi}(s_{t+1}^{k}, a_{t+1}^{'k})$最大值的那个$a_{t+1}^{'k}$；
    * 将$s_{t+1}^{k}$和$a_{t+1}^{k}$作为输入，输入到$\hat{Q}_{\pi}$中，求出$r_{t+1}^{k} + q_{\pi}(s_{t+1}^{k}, a_{t+1}^{k})$；
    * 由$q_{\pi}(s_{t}^{k}, a_{t}^{k} = r_{t+1}^{k} + q_{\pi}(s_{t+1}^{k}, a_{t+1}^{k})$，将$Q_{\pi}$网络输出减去$\hat{Q}_{\pi}$网络的输出，得到$r_{t+1}$；
    * 我们的Experience Buffer元组中的$r_{t+1}$为Ground Truth的值，运行回归算法（Regression），更新$Q_{\pi}$网络的参数；
    * 当运行$C$次时，用$Q_{\pi}$网络的参数更新$\hat{Q}_{\pi}$网络的参数；
    
以上就是传统的DQN算法基本原理，但是在实际应用中，DQN会显著高估$q_{\pi}(s,a)$的值，为了解决这一问题，人们引入了Double DQN。

### 1.4.5. Double DQN算法
Double DQN算法可以视为Double DQN + Prior Sampling + MC / TD Balance的集成，具体过程如下所示：
* 初始化NN和Target NN：初先初始化NN网络$Q_{\pi}$和Target NN网络$\hat{Q}_{\pi}$，并且令二者的参数相等，即$Q_{\pi}=\hat{Q}_{\pi}$；
* 让Agent与环境进行$K$个Episode：
  * 在第$k$个Episode的第$t$个时间点：
    * 其在$1-\epsilon$的概率下使用下式选出的行动，在$\epsilon$概率下随机选择：
    $$
    a_{t}^{k} = \arg \max_{ a_{t}^{k} \in \mathcal{A}(s_{t}^{k}) } q_{\pi}(s_{t}^{k}, a_{t}^{k})
    $$
    实际的求解方法为将$s_{t}^{k}$与所有的$a_{t}^{'k} \in \mathcal{A}(s_{t}^{k})$的行动对作为输入，输入到$Q_{\pi}$网络中，计算出相应的$q_{\pi}(s_{t}^{k}, a_{t}^{'k})$的值，最终的$a_{t}^{k}$为其中可以取得$q_{\pi}(s_{t}^{k}, a_{t}^{'k})$最大值的那个$a_{t}^{'k}$；
    * 重复上面的步聚$N$（超参数）次，第$N$次时，状态为$s_{t+N}^{k}$，采取的行动为$a_{t+N}^{k}$，会得到奖励$r_{t+N+1}^{k}$，转移到新状态$s_{t+N+1}^{k}$；
    * 将元组$s_{t}^{k}, a_{t}^{k}, r_{t+1}^{k}, s_{t+1}^{k}, ..., s_{t+N}^{k}, a_{t+N}^{k}, r_{t+N+1}^{k}, s_{t+N+1}^{k}$保存到Experience Buffer中；
    * 如果Experience Buffer小于batch_size大小，则直接返回内层循环顶部，进行第$k$个episode的第$t+1$时刻。如果Experience Buffer已经达到最大容量，则将最老的元组挤出，将当前元组加入；
    * 从Experience Buffer中Sample出batch_size个元组，在实际应用中，可以专门Sample效果不好的元组；
    * 将$s_{t+N+1}^{k}$与所有行动$a_{t+N+1}^{'k} \in \mathcal{A}(s_{t+N+1}^{k})$组成状态行动对，将每个状态行动对输入到网络$Q_{\pi}$中，得到$q_{\pi}(s_{t+N+1}^{k}, a_{t+N+1}^{'k})$中取最大值时的$a_{t+N+1}^{'k}$作为$a_{t+N+1}^{k}$；
    * 将$s_{t+N+1}^{k}$和刚求出的$a_{t+N+1}^{k}$输入到网络$\hat{Q}_{\pi}$中，得到：
    $$
    y^{k} = \sum_{i=t}^{t+N+1} r_{i} + q_{\pi}(s_{t+N+1}^{k}, a_{t+N+1}^{k})
    $$
    * 将$s_{t}$和$a_{t}$输入到网络$Q_{\pi}$中，得到$q_{\pi}(s_{t}, a_{t})$，将$y^{k}$作为Ground Truth的值训练$Q_{\pi}$网络，更新其参数；
    * 当重复更新网络$Q_{\pi}$参数$C$（超参数）时，用网络$Q_{\pi}$的参数更新网络$\hat{Q}_{\pi}$的参数；
 在上面的
